In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import concurrent.futures
import pandas as pd
import itertools
import time

# Exercise 17: Simulate Gaussian profiles with multiple changepoints 

In [ ]:
def simulate_mcp_data(n, tau, mu, sigma=1.0, random_seed=None):
    """
    Simulate i.i.d. Gaussian data with several changes in the mean.
    
    Parameters:
      n           : int
                        Total number of observations.
      tau         : list or array-like
                        Positions (indices) where the mean changes.
                        Each change position must be in {1, 2, ..., n-1} and the list must be sorted in increasing order.
      mu          : list or array-like
                        Mean values for each segment. The length must equal len(change_positions) + 1.
      sigma       : float (default=1.0)
                        Standard deviation (assumed constant across all segments).
      random_seed : int or None (default=None)
                        Seed for reproducibility.
      
    Returns:
      mu   : numpy array of shape (n,)
             The true mean vector (each segment filled with its corresponding mean).
      data : numpy array of shape (n,)
             The simulated data: true mean + Gaussian noise.
    """
    if random_seed is not None:
        np.random.seed(random_seed)
    
    # Ensure change_positions is sorted and valid
    tau = sorted(tau)
    if any(t < 1 or t >= n for t in tau):
        raise ValueError("Each change position must be in {1, 2, ..., n-1}.")
    
    if len(mu) != len(tau) + 1:
        raise ValueError("The number of segment means must equal the number of change positions plus one.")
    
    tau   = np.concatenate(([0], tau, [n]))
    mu    = np.repeat(mu,tau[1:]-tau[:-1]) 
    noise = np.random.normal(loc=0, scale=sigma, size=n)
    data  = mu + noise
    
    return mu, data

In [ ]:
# Example: simulate data with 5 changepoints
n   = 200
tau = [10, 70, 100, 150, 180]
mu  = [0.0, 2.0, -1.0, 3.0, 0.0, 1.0]

mu, y = simulate_mcp_data(n, tau, mu, sigma=1.0, random_seed=42)

# Plot the simulated data and the true underlying mean
x = np.arange(1, n+1)

plt.figure(figsize=(10, 5))
plt.plot(x, y, label='Simulated data', color='gray', alpha=0.7)
plt.step(x, mu, label='Mean vector', color='blue', linewidth=1)
for cp in tau:
    plt.axvline(x=cp, color='red', linestyle='--', label=f'Changepoint' if cp==tau[0] else None)
plt.xlabel('Observation Index')
plt.ylabel('Value')
plt.title('Simulated Gaussian Data with Multiple Changepoints')
plt.legend()
plt.show()

# Exercise 21 : Generic OP algorithm

In [ ]:
from abc import ABC, abstractmethod

################################################################################
# 1) Abstract base class for Cost Functions
################################################################################

class CostFunction(ABC):
    """
    Abstract base class: any cost function used by OptimalPartitioning 
    must implement these methods.
    """

    @abstractmethod
    def precompute_summaries(self, data: np.ndarray, weights: np.ndarray = None):
        """
        Perform one-time computations (e.g. cumsums) needed to quickly 
        evaluate segment costs. Some cost functions may also handle 'weights'.
        """
        pass

    @abstractmethod
    def cost(self, start: int, end: int) -> float:
        """
        Returns the cost (negative log-likelihood or sum of squares, etc.)
        for the segment data[start:end+1].
        Indices are inclusive, 0-based.
        """
        pass

################################################################################
# 2) Concrete cost classes
################################################################################

class GaussianCPInMeanCost(CostFunction):
    """
    Example for a Weighted Gaussian change-in-mean model:
      cost(segment) = sum( w_i * (y_i - mu)^2 ).
    We store cumsum(w), cumsum(w*y), cumsum(w*y^2).
    """

    def __init__(self):
        self.cum_w     = None
        self.cum_wy    = None

    def precompute_summaries(self, data: np.ndarray, weights: np.ndarray = None):
        if weights is None:
            # default to all weights = 1
            weights = np.ones_like(data)
        
        n = len(data)
        
        self.cum_w  = np.concatenate(([0], np.cumsum(weights)))
        self.cum_wy = np.concatenate(([0], np.cumsum(weights*data)))

    def cost(self, start: int, end: int) -> float:
        w_seg  = self.cum_w[end]   - self.cum_w[start]
        wy_seg = self.cum_wy[end]  - self.cum_wy[start]

        return - (wy_seg**2)/w_seg

################################################################################
# 3) The OptimalPartitioning class with min seg length
################################################################################

class OptimalPartitioning:
    """
    Generic implementation of the Optimal Partitioning (OP) algorithm, with:
      - penalty (beta)
      - any cost function implementing CostFunction
      - minimum segment length
    """

    def __init__(self, 
                 penalty: float, 
                 cost_func: CostFunction, 
                 min_seg_len: int = 1):
        """
        Parameters
        ----------
        penalty : float
            The penalty (beta) for each new segment.
        cost_func : CostFunction
            An instance implementing cost() and precompute_summaries().
        min_seg_len : int
            The minimum allowable segment length.
        """
        self.penalty         = penalty
        self.cost_func       = cost_func
        self.min_seg_len     = min_seg_len

        # Will be filled after calling .fit():
        self._L   = None
        self._cp  = None
        self.n    = 0
        self.data = None
        self.changepoints = None   # final result

    def fit(self, data: np.ndarray, weights: np.ndarray = None):
        """
        Run the dynamic programming recursion. 
        data: the 1D time series. 
        weights: optional array of same shape as data (for Weighted cost).
        """
        self.data = data
        self.n    = len(data)

        # 1) Precompute cost function summaries
        self.cost_func.precompute_summaries(data, weights)

        # 2) Initialize DP arrays
        L    = np.full(self.n+1, np.inf, dtype=float)
        cp   = np.zeros(self.n+1, dtype=int)
        L[0] = -self.penalty  # so that for t=1, it cancels out

        # 3) Main recursion
        for t in range(1, self.n+1):
            best_val = np.inf
            best_tau = 0

            for tau in range(0, t - self.min_seg_len + 1):
                val = L[tau] + self.cost_func.cost(tau, t) + self.penalty
                if val < best_val:
                    best_val = val
                    best_tau = tau

            L[t]  = best_val
            cp[t] = best_tau

        # 4) Save DP arrays
        self._L  = L
        self._cp = cp

        # 5) Backtrack to get changepoints
        changepoints = []
        curr = self.n
        while curr > 0:
            tau_star = cp[curr]
            # record the boundary (if > 0)
            if tau_star > 0:
                changepoints.append(tau_star)
            curr = tau_star
        changepoints.reverse()
        self.changepoints = changepoints

    def get_changepoints(self):
        return self.changepoints
    
    def get_min_cost(self):
        """
        The minimal cost of segmenting the entire dataset (index 0..n-1).
        """
        return self._L[-1] if self._L is not None else None

In [ ]:
################################################################################
# OP on a profile (n~3000), with 3 changepoints

np.random.seed(123)
len_seg = 2**10
data = np.concatenate([
    np.random.normal(0.0, 1.0, len_seg), 
    np.random.normal(2.0, 1.0, len_seg), 
    np.random.normal(0.0, 1.0, len_seg)
])

op = OptimalPartitioning(
    penalty = 2*np.log(len(data)), 
    cost_func = GaussianCPInMeanCost()
)
op.fit(data)

print("Changepoints:", op.get_changepoints())
print("Min cost:", op.get_min_cost())